In [11]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import yfinance as yf
import mplfinance as mpf
import requests
import plotly.graph_objects as go
import talib

In [12]:
def visualize(df, title="TSLA Candlestick Chart"):
    """
    Plots a candlestick chart using mplfinance for the given DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing OHLC data.
    title (str): The title of the chart.
    """
    # Ensure the DataFrame index is a datetime index
    df = df.copy()
    df.index = pd.to_datetime(df.index)

    # Plot the candlestick chart
    mpf.plot(df, type='candle', style='charles', title=title, ylabel="Price",
             volume=True, figsize=(10, 5))

def visualize_interactive(df, title="TSLA Candlestick Chart"):
    """
    Plots an interactive candlestick chart using Plotly.

    Parameters:
    df (pd.DataFrame): The DataFrame containing OHLC data.
    title (str): The title of the chart.
    """
    fig = go.Figure(data=[go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='TSLA'
    )])

    fig.update_layout(title=title, xaxis_rangeslider_visible=False)
    fig.show()

def get_tsla_recent_data():
    session = requests.Session()
    session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
    tsla = yf.Ticker('TSLA', session=session)
    df = tsla.history(period='4mo', interval='1d')
    return df

def detect_engulfing_pattern(tsla_df):
    tsla_df['Engulfing'] = ta.cdl_pattern(tsla_df['Open'], tsla_df['High'], tsla_df['Low'], tsla_df['Close'], name="engulfing")
    return tsla_df

def visualize_engulfing_pattern(tsla_df):
    # Create the candlestick chart
    fig = go.Figure(data=[go.Candlestick(
        x=tsla_df.index,
        open=tsla_df['Open'],
        high=tsla_df['High'],
        low=tsla_df['Low'],
        close=tsla_df['Close'],
        name='TSLA'
    )])

    # Highlight Engulfing Patterns
    engulfing_candles = tsla_df[tsla_df['Engulfing'] != 0]
    
    # Separate Bullish and Bearish Engulfing Patterns
    bullish_engulfing = tsla_df[tsla_df['Engulfing'] == 100]
    bearish_engulfing = tsla_df[tsla_df['Engulfing'] == -100]
    
    # Mark Bullish Engulfing Patterns (Green Triangle-Up)
    fig.add_trace(go.Scatter(
        x=bullish_engulfing.index,
        y=bullish_engulfing['High']+10,
        mode='markers',
        marker=dict(size=10, color='green', symbol='triangle-up'),
        name='Bullish Engulfing'
    ))

    # Mark Bearish Engulfing Patterns (Red Triangle-Down)
    fig.add_trace(go.Scatter(
        x=bearish_engulfing.index,
        y=bearish_engulfing['High']+10,
        mode='markers',
        marker=dict(size=10, color='red', symbol='triangle-down'),
        name='Bearish Engulfing'
    ))

    # Chart Layout
    fig.update_layout(
        title="TSLA Candlestick Chart with Engulfing Patterns",
        xaxis_title="Date",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False
    )

    # Show the figure
    fig.show()

In [13]:
tsla_data = get_tsla_recent_data()

In [14]:
tsla_data.to_csv("engulfing_patterns.csv", index=True)  # Saves with index (Date)

In [15]:
#visualize(tsla_data)
visualize_interactive(tsla_data)

In [16]:
#Run the function and get Engulfing Pattern data
engulfing_patterns = detect_engulfing_pattern(tsla_data)

engulfing_df = engulfing_patterns[engulfing_patterns['Engulfing'] != 0]
engulfing_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Engulfing
Date,,,,,,,,
2024-11-19 00:00:00-05:00,335.760010,347.380005,332.750000,346.000000,88852500,0.0,0.0,100.0
2024-11-25 00:00:00-05:00,360.140015,361.929993,338.200012,338.589996,95890900,0.0,0.0,-100.0
2025-02-04 00:00:00-05:00,382.630005,394.000000,381.399994,392.209991,57072200,0.0,0.0,100.0
2025-02-19 00:00:00-05:00,354.000000,367.339996,353.670013,360.559998,67094400,0.0,0.0,100.0
2025-02-28 00:00:00-05:00,279.500000,293.880005,273.600006,292.980011,115697000,0.0,0.0,100.0


### Displaying markers

In [17]:
visualize_engulfing_pattern(tsla_data)